In [1]:
import pandas as pd

from notebooks.ford.decode import search, print_breakdown


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

df_nhtsa = search(
  # searches=['Bronco Sport', 'Escape'],
  # searches=['Explorer'],
  # searches=['F-150'],
  min_model_year=2018,  # one year before the first supported
  include_openpilot=True,
  include_police=False,
)

print()
print_breakdown(df_nhtsa)

Loaded 2873 VINs (filter_comment=None, include_openpilot=True, skipped=78)


Loaded AsBuilt data for 2873 VINs


Decoding VINs: 100%|██████████| 2873/2873 [00:00<00:00, 19009.26it/s]


Decoded 2873 VINs

Model
Aviator             73
Bronco              45
Bronco Sport       219
Continental          4
Corsair             51
Ecosport            12
Edge               313
Escape             530
Expedition          45
Expedition MAX      55
Explorer           569
F-150              278
F-250               19
F-350               13
F-550                1
Fiesta              51
Focus               48
Fusion              10
MKC                  1
MKZ                 28
Maverick           177
Mustang             26
Mustang Mach-E      97
Nautilus            24
Navigator           27
Navigator L         17
Ranger              12
Transit             55
Transit Connect     45
dtype: int64

Model            ModelYear
Aviator          2020          27
                 2021           9
                 2022          12
                 2023          24
                 2024           1
Bronco           2021           5
                 2022           3
                 2023        

In [2]:
SKIP = [
  'AirBagLocFront',
  'AirBagLocKnee',
  'AirBagLocSide',
  'BodyClass',
  'DisplacementCC',
  'DisplacementCI',
  'DisplacementL',
  'EngineCylinders',
  'GVWR',
  'LowerBeamHeadlampLightSource',
  'Make',
  'MakeID',
  'Manufacturer',
  'ManufacturerId',
  'ModelID',
  'NCSABodyType',
  'NCSAMake',
  'NCSAModel',
  'PlantCity',
  'PlantCompanyName',
  'PlantCountry',
  'PlantState',
  'Seats',
  'WheelSizeFront',
  'WheelSizeRear',
  'VehicleType',
  'VIN',
  'VehicleDescriptor',
  'WheelSizeFront',
  'WheelSizeRear',
]

KEEP = [
  # 'DisplacementL',
  'DriveType',
  # 'ElectrificationLevel',
  # 'FuelTypePrimary',
  # 'FuelTypeSecondary',
  'Model',
  'Series',
  'Trim',
]

properties = {}

for col in KEEP:
  if col not in df_nhtsa.columns:
    print(f'WARNING: {col} not in df_nhtsa.columns')

for col in df_nhtsa.columns:
  if col in SKIP:
    continue

  property_values = set(df_nhtsa[col].unique())
  if col not in KEEP:
    if '' in property_values:
      continue
    if len(property_values) == 1:
      continue

  properties[col] = property_values

df_nhtsa.drop(columns=['FuelTypePrimary', 'FuelTypeSecondary'], inplace=True, errors='ignore')
properties.pop('FuelTypePrimary', None)
properties.pop('FuelTypeSecondary', None)

properties

{'DriveType': {'2WD', '4WD', 'AWD', 'FWD', 'RWD', 'Unknown'},
 'ElectrificationLevel': {'BEV', 'FHEV', 'HEV', 'ICE', 'PHEV'},
 'Model': {'Aviator',
  'Bronco',
  'Bronco Sport',
  'Continental',
  'Corsair',
  'Ecosport',
  'Edge',
  'Escape',
  'Expedition',
  'Expedition MAX',
  'Explorer',
  'F-150',
  'F-250',
  'F-350',
  'F-550',
  'Fiesta',
  'Focus',
  'Fusion',
  'MKC',
  'MKZ',
  'Maverick',
  'Mustang',
  'Mustang Mach-E',
  'Nautilus',
  'Navigator',
  'Navigator L',
  'Ranger',
  'Transit',
  'Transit Connect'},
 'ModelYear': {2018, 2019, 2020, 2021, 2022, 2023, 2024},
 'Series': {'',
  '150',
  '250',
  '350',
  '350 HD',
  'AWD MKZ Reserve I',
  'Active',
  'BADLANDS',
  'BASE',
  'BIG BEND',
  'BL',
  'Badlands',
  'Base',
  'Base, Big Bend, Black Diamond, Outer Banks',
  'Base, Big Bend, Black Diamond, Outer Banks, Badlands, Wildtrak',
  'Base, Big Bend, Black Diamond, Outer Banks, Badlands, Wildtrak, Everglades',
  'Base/Big Bend/Black Diamond/Outer Banks',
  'Base/Bi

### Combine NHTSA and Ford AsBuilt Data

We fetch the factory part numbers (software and hardware) from the Ford AsBuilt data and combine it with the NHTSA data.

In [3]:
from panda.python.uds import DATA_IDENTIFIER_TYPE

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.ecu import FordEcu
from notebooks.ford.settings import VehicleSetting, VehicleSettings


df_fw = df_nhtsa.copy()

# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
)


ecus = {
  'abs': FordEcu.AntiLockBrakeSystem,
  # 'engine': FordEcu.PowertrainControlModule,
  'eps': FordEcu.PowerSteeringControlModule,
  'fwdCamera': FordEcu.ImageProcessingModuleA,
  'fwdRadar': FordEcu.CruiseControlModule,
}


def get_ecu_identifier(ecu: FordEcu, identifier: int):
  def apply(row):
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return ''
    return data.get_identifier(ecu, identifier)
  return apply


def get_setting(setting: VehicleSetting):
  def apply(row):
    ecu = setting.ecu
    ecu_name = next(filter(lambda name: ecus[name] == ecu, ecus.keys()))
    ecu_part = row[f'{ecu_name}_part']
    if not ecu_part:
      return 'Missing ECU'
    # TODO: detect ECUs which we can't read
    # if ecu_part[0] in ('M', 'N', 'O', 'P'):
    #   return 'CAN FD'
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return 'Missing ECU'
    return data.get_setting_value(setting)
  return apply


# Add the ECU identifiers
for name, ecu in ecus.items():
  df_fw[f'{name}_fw'] = df_fw.apply(
    get_ecu_identifier(
      ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER
    ),
    axis=1,
  )
  df_fw[f'{name}_part'] = df_fw.apply(get_ecu_identifier(ecu, 0xF111), axis=1)


# Determine platform (pre-Q3, Q3 or Q4)
def get_platform(row):
  camera_pn = row['fwdCamera_part']
  core = camera_pn.split('-')[1] if camera_pn else None
  if not core:
    return 'No camera'

  platform = {
    '14F403': 'Q3',
    '14G025': 'pre-Q3',  # guess, seen on 2020 Fusion/Mondeo
    '14H107': 'Q4',
  }.get(core)

  if platform:
    return platform
  assert False, f'Unhandled platform for {row["ModelYear"]} {row["Model"]} {camera_pn=}'


df_fw['Platform'] = df_fw.apply(get_platform, axis=1)


# Add settings
settings = {
  'ACC (IPMA)': VehicleSettings.ipma_enable_adaptive_cruise,
  'TJA (IPMA)': VehicleSettings.ipma_enable_traffic_jam_assist,
}
for name, setting in settings.items():
  df_fw[name] = df_fw.apply(get_setting(setting), axis=1)


# Clear settings if platform is Q4
# TODO: simplify
for setting in settings.keys():
  df_fw[setting] = df_fw.apply(
    lambda row: '' if row['Platform'] == 'Q4' else row[setting],
    axis=1,
  )


# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
  errors='ignore',
)


# Merge 'Series' and 'Trim' properties (they should be mutually exclusive)
if 'Trim' in properties and 'Series' in properties:
  def get_series_or_trim(row):
    series, trim = row['Series'], row['Trim']
    if series == 'F-Series':
      return trim
    elif series and trim:
      return f'{series} ({trim})'
      # raise ValueError(f'{row["VIN"]} Both Series and Trim are set: {series} and {trim}')
    return series or trim

  df_fw['Trim'] = df_fw.apply(get_series_or_trim, axis=1)
  df_fw.drop(columns=['Series'], inplace=True)
  properties.pop('Series', None)
elif 'Series' in properties:
  df_fw.rename(columns={'Series': 'Trim'}, inplace=True)
  properties['Trim'] = properties.pop('Series')


# Drop columns that are all empty
df_fw = df_fw.loc[:, (df_fw != '').any(axis=0)]


# Drop the VIN
df_fw.drop(columns=['VIN'], inplace=True, errors='ignore')


# Sort by columns
main_columns = []
if 'ModelYear' in df_fw.columns:
  main_columns.append('ModelYear')
if 'Series' in df_fw.columns:
  main_columns.append('Series')

extra_columns = list(set(properties.keys()) - set(main_columns))

# + [f'code_{name}' for name in settings.keys()] 
df_fw.sort_values(
  by=main_columns + extra_columns + [f'{name}_fw' for name in ecus.keys()],
  ascending=False,
  inplace=True,
  ignore_index=True,
)

# Delete 'Series' column
df_fw.drop(columns=['Series'], inplace=True, errors='ignore')


# Drop columns that are all the same
# df_fw = df_fw.loc[:, df_fw.apply(pd.Series.nunique) != 1]


# Add asterisks to column name where the value is the same for all rows
# df_fw.rename(
#   columns={
#     col: f'*{col}' if len(set(df_fw[col].unique())) == 1 else col
#     for col in df_fw.columns
#   },
#   inplace=True,
# )

# Remove duplicate rows

count = len(df_fw)
df_fw.drop_duplicates(inplace=True)
print(f'Removed {count - len(df_fw)} duplicate rows')


df_fw

Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AC', 61715: 'NL38-10D678-YC', 61732: 'NL38-10D682-AL', 61832: 'NL38-10D681-AL', 61836: 'DVMUA22354J01966'}
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3C-14E098-BB', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002312882', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AC', 61715: 'NL38-10D678-YC', 61732: 'NL38-10D682-AL', 61832: 'NL38-10D681-AL', 61836: 'DVMUA23021J02516'}
Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AB', 61715: 'NL38-10D678-YB', 61732: 'NL38-10D682-AL', 61832: 'NL38-10D681-AL', 61836: 'DVMUA21351J00019'}
Unknown ECU address: 224 (0xe0)
0xe0 {61713: 'LC3A-14D607-BC', 61715: 'LC3A-12B533-BC', 61832: 'LC3A-14D609-BC', 61836: '0168393649051346'}
Unknown ECU address: 229 (0xe5)
0xe5 {61713: 'KC3A-5M

,DriveType,ElectrificationLevel,Model,ModelYear,Trim,abs_fw,abs_part,eps_fw,eps_part,fwdCamera_fw,fwdCamera_part,fwdRadar_fw,fwdRadar_part,Platform,ACC (IPMA),TJA (IPMA)
0,AWD,ICE,Maverick,2024,XLT TREMOR,PZ6C-2D053-GE,PZ6C-14F065-GA,NZ6C-14D003-AL,LX6C-14F079-BB,NZ6T-14F397-AC,NZ6T-14F403-CA,,,Q3,Off,Disabled
3,RWD,ICE,Explorer,2024,XLT,L1MC-2D053-KB,L1MC-14F065-DA,P1MC-14D003-AA,P1MC-14F079-AA,LB5T-14F397-AF,LB5T-14F403-CA,LB5T-14D049-AB,LB5T-14F089-AA,Q3,RadarFusion,Enabled
4,RWD,ICE,Explorer,2024,XLT,L1MC-2D053-KB,L1MC-14F065-DA,P1MC-14D003-AA,P1MC-14F079-AA,LB5T-14F397-AF,LB5T-14F403-CA,,,Q3,Off,Disabled
6,FWD,ICE,Maverick,2024,XLT,PZ6C-2D053-GE,PZ6C-14F065-GA,NZ6C-14D003-AL,LX6C-14F079-BB,NZ6T-14F397-AC,NZ6T-14F403-CA,,,Q3,Off,Disabled
13,FWD,HEV,Maverick,2024,XLT,PZ6C-2D053-EF,PZ6C-14F065-EA,NZ6C-14D003-AL,LX6C-14F079-BB,NZ6T-14F397-AC,NZ6T-14F403-CA,,,Q3,Off,Disabled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839,FWD,ICE,Focus,2018,S,F1FC-14C036-AJ,F1FC-14F067-AB,HV6T-14C217-AA,JV6T-14C262-AB,,,,,No camera,Missing ECU,Missing ECU
2840,FWD,ICE,Fiesta,2018,S,D2BC-2D053-FC,D2BC-14F065-FA,AE81-14C217-GD,None,,,,,No camera,Missing ECU,Missing ECU
2841,2WD,ICE,Escape,2018,S,GV61-14C036-CJ,GV61-14F067-CC,HV6T-14C217-AA,JV6T-14C262-AB,,,,,No camera,Missing ECU,Missing ECU
2843,4WD,ICE,Expedition,2018,Platinum,JL14-2D053-AE,JL34-14F065-AC,JL14-14D003-CA,JR3C-14F079-AA,JL1T-14F397-AF,JL1T-14F403-CB,H1BT-14D049-AF,H1BT-14F089-AC,Q3,RadarFusion,Disabled


In [4]:
df_fw_analysis = df_fw.copy()

SORTED_TRIM_LEVELS = [
  'Base',
  # Escape, Focus, Fusion
  'S',
  'SE',
  'SEL',
  'Titanium',
  'Active',
  'ST Line',
  'ST Line Select',
  'ST Line Elite',
  'ST Line Premium',
  'PHEV',
  # Bronco Sport
  'BIG BEND',
  'OUTER BANKS',
  'BADLANDS',
  'HERITAGE',
  # Explorer
  'XL',
  'XLT',
  'Limited',
  'ST',
  'Platinum',
  'Timberline',
  'King Ranch',
  # Aviator
  'Standard',
  'Reserve',
  'Livery',
  'Grand Touring',
]

def sort_trim_level(trim):
  if trim in SORTED_TRIM_LEVELS:
    return SORTED_TRIM_LEVELS.index(trim)
  return 1000

results = dict()

for ecu in reversed(ecus.keys()):
  print(f'# Ecu.{ecu}')

  fw_groups = df_fw_analysis \
    .drop(
      columns=[f'{name}_fw' for name in ecus.keys() if name != ecu] + [f'{name}_part' for name in ecus.keys() if name != ecu],
    ) \
    .rename(
      columns={
        f'{ecu}_fw': 'fw',
        f'{ecu}_part': 'part',
      },
    ) \
    .groupby(
      by=['part', 'fw'],
      dropna=False,
    )

  results[ecu] = fw_groups.agg(lambda x: ', '.join(map(lambda x: str(x), set(x))))
  print(results[ecu].to_string())

  print()

with pd.ExcelWriter('ford_fw.xlsx', engine='xlsxwriter') as writer:
  for ecu, result in results.items():
    result.to_excel(writer, sheet_name=ecu)

# Ecu.fwdRadar
                                                      DriveType       ElectrificationLevel                                                                                                                                                                                                                        Model                                 ModelYear                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        